# Study of Anca Radulescu 2008 paper on computing topological entropy in a space of quartic polynomials

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
from functools import reduce

Let $q_\lambda:[0,1] \rightarrow [0,1]$ be a logistic function with parameter $\lambda \in [0,4]$, so $q_\lambda(x) = \lambda x(1-x)$.  Let $P^Q$ be the parameter space of quartic polynomials which are a composition of two logistic functions.
The algorithm in question aims to estimate the topological entropy of a quartic polynomial $g:I \rightarrow I$, where $I = [0,1]$.  Fix a parameter $(\lambda,\mu) \in P^Q$.  We will estimate the entropy of the quartic map $g = q_\mu \circ q_\lambda$. 

Let $P^T$ be the parameter space of the tent maps of the form $T_a(x) = a(1 - |x - 1|)$ such that $0 \leq a \leq 2$, and let $P^{ST}$ be the space of stunted sawtooth maps.  We let $U^T = \{(s,t) \in [-\log 2, \log 2]^2, s + t \geq 0\}$.

The isentrope $i^T(h^*) = \{(s,t) \in U^T, s + t = h^*\}$, and has boundary points $L = (s_l,t_l) = (h^* - \log 2, \log 2)$, $R = (s_R,t_R) = (\log 2, h^* - \log 2)$.

We start with an underestimate $h_0$ and an overestimate $h_1$ for $h(g)$.  We will initialise them as $h_0 = 0$ and $h_1 = 4.1$.  We wish to improve these estimates, and will use an iterated bisection technique.  We wish to do this until we achieve an error range of $|h_1 - h_0| < \varepsilon = 10^{-4}$.  Then the algorithm stops, returning the final underestimate and final overestimate of $h_0(g) = h_0$ and $h_1(g) = h_1$ respectively. 

The first step is to compute its pair of critical d'itineraries. 

First we generate its diorbit, up to the $N$-th iteration of $f_1$ and $f_2$.

In [ ]:
def diorbit(f1,f2,x,N):
    #This function calculates the diorbit of x under (f1,f2).
    L = [x]
    for _ in range(N):
        L += [f1(L[-1])]
        L += [f2(L[-1])]
    return L

Now we define the d'itinerary of a point $x$ under the functions $f_1: I_1 \rightarrow I_2$ and $f_2: I_2 \rightarrow I_1$, which is a sequence of letters from the following alphabets $\{L_1,C_1,R_1\}$ and $\{L_2,C_2,R_2\}$, where $L_1$ and $R_1$ are the subintervals of points that are left and right of the critical point of $f_1$, respectively, and $C_1 = \{c_1\}$, where $c_1$ is the critical point of $f_1$. This is similar for the second alphabet.  Note that if $x \in I_1$, the d'itinerary starts with $(x, f_1(x), f_2(f_1(x)), ...)$, and if $x \in I_2$, the d'itinerary starts with $(x, f_2(x), f_1(f_2(x)), ...)$.

In [ ]:
def ditinerary(f1,f2,x,N):
    #This function calculates the d'itinerary of x under (f1,f2).  Note that the ditinerary of (f1,f2) is distinct to that of (f2,f1)
    O = diorbit(f1,f2,x,N)
    
    L = []
    for n in range(N):
        #We know that the ditinerary involving alternating choices of f1 and f2, so we simply do both in one step.
        #f1
        if round(O[n],50) < 0.5:
            L += ['L1']
        elif round(O[n],50) == 0.5:
            L += ['C1']
        elif round(O[n],50) > 0.5:
            L += ['R1']
        #f2
        if round(O[n+1],50) < 0.5:
            L += ['L2']
        elif round(O[n+1],50) == 0.5:
            L += ['C2']
        elif round(O[n+1],50) > 0.5:
            L += ['R2']
    
    return L

Now we will create a function which represents the partial order on itineraries.  Here, we use -1 to indicate that the first input is less than the second, 0 to indicate that they're equal, and 1 to indicate that the first input is greater than the second.

In [ ]:
#Functions determining the sign of a ditinerary based on Radulescu 2008
def letterSign(A):
    if A == 'L1' or A == 'L2':
        return 1
    elif A == 'C1' or A == 'C2':
        return 0
    elif A == 'R1' or A == 'R2':
        return -1

def itSign(I):
    return [letterSign(x) for x in I[1:]]

def letterOrder(A,B):
    if A == B:
        return 0
    elif A == 'L1' or (A == 'C1' and B == 'R1') or A == 'L2' or (A == 'C2' and B == 'R2'):
        return -1
    elif B == 'L1' or (B == 'C1' and A == 'R1') or B == 'L2' or (B == 'C2' and A == 'R2'):
        return 1

def itineraryOrder(I1,I2):
    if I1 == I2:
        return 0
    
    k = 0
    while I1[k] == I2[k] and k < len(I1):
        k += 1
    
    S = itSign(I1)
    
    if k == 0:
        return letterOrder(I1[0],I2[0])
    
    if len(S) > 1:
        multiSign = reduce(lambda a,b : a*b, S[0:k-1], 1) #First element of itSign will be 0 on the critical d'itineraries, so we omit it.
    elif len(S) == 1:
        multiSign = S[0]
    #print(letterOrder(I1[k],I2[k]), multiSign)
    if letterOrder(I1[k],I2[k])*multiSign == 1:
        return 1
    elif letterOrder(I1[k],I2[k])*multiSign == -1:
        return -1

Of course, it's not guaranteed that two itineraries or kneading data are comparable.  However, it is proven that given a map $f = q_\mu \circ q_\lambda$, with $(\mu,\lambda) \in P^Q$, and any given isentrope $i^T(h^*)$, there exists a point $C \in i^T(h^*)$ such that the map $g_C$ has kneading data comparable with the kneading data of $f$, i.e. $K(f)$.  The algorithm to find such a map is an iterated bisection technique.

Let $c_1$ and $c_2$ be the critical points of $q_\lambda$ and $q_\mu$, respectively.  Let $m_1$ and $m_2$ be the d'itineraries of $c_1$ and $c_2$ under $(q_\lambda,q_\mu)$.

Given a $h^*$-isentrope, we first check the endpoints $L$ and $R$.  For $X \in i^T(h^*)$, the critical d'itineraries of $X$ are denoted $(k_1(X),k_2(X))$. The kneading data of $g_X$ is comparable with $K(f)$ iff:
\begin{equation}
\text{either  } \Bigg\{ \begin{aligned}
                       k_1(X) \leq m_1 \\
                       k_2(X) \leq m_2
                       \end{aligned} \text{  or  } \Bigg\{ \begin{aligned}
                       k_1(X) \geq m_1 \\
                       k_2(X) \geq m_2
                       \end{aligned}
\end{equation}

In [ ]:
def MapComparability(k1,k2,m1,m2):
    #This function checks whether two maps are comparable up to a certain point of their itinerary, or if the comparable map is above or below in the line of parameters.
    k1m1 = itineraryOrder(k1,m1)
    k2m2 = itineraryOrder(k2,m2)
    #print(k1,'\n',m1)
    
    #print(k2,'\n',m2)
    if (k1m1 == -1 or k1m1 == 0) and (k2m2 == -1 or k2m2 == 0):
        return 'K(M) <= K(f)'
    elif (k1m1 == 1 or k1m1 == 0) and (k2m2 == 1 or k2m2 == 0):
        return 'K(M) >= K(f)'
    elif k1m1 == 1 and k2m2 == -1:
        return 'Better upper bound'
    elif k1m1 == -1 and k2m2 == 1:
        return 'Better lower bound'

In [ ]:
def FindComparableMap(q1,q2,h,N):
    #This function finds a comparable sawtooth map given quadratics q1 and q2, and an isentrope at h
    n = 10
    #calculate ditineraries
    m1 = ditinerary(q1,q2,0.5,n)
    m2 = ditinerary(q2,q1,0.5,n)
    
    #lower and upper bounds
    L = np.asarray([h - np.log(2),np.log(2)]) - 1e-2
    R = np.asarray([np.log(2),h - np.log(2)]) + 1e-2
    
    fK = lambda K : lambda x : K*(0.5 - abs(x - 0.5))
    k = lambda M : [ditinerary(fK(M[0]),fK(M[1]),0.5,n),ditinerary(fK(M[1]),fK(M[0]),0.5,n)]
    
    #We find comparable maps for the L and R parameters
    Lcomp = MapComparability(k(np.exp(L))[0],k(np.exp(L))[1],m1,m2)
    Rcomp = MapComparability(k(np.exp(R))[0],k(np.exp(R))[1],m1,m2)
    
    #If it is immediately comparable, then we are done
    if Lcomp == 'K(M) <= K(f)' or Lcomp == 'K(M) >= K(f)':
        return Lcomp
    if Rcomp == 'K(M) <= K(f)' or Rcomp == 'K(M) >= K(f)':
        return Rcomp
    
    for _ in range(N):
        #If not immediately comparable, then we choose the midpoint, compare, and then set the midpoint to be either the new lower bound or new upper bound
        M = (L + R)/2
        Mcomp = MapComparability(k(np.exp(M))[0],k(np.exp(M))[1],m1,m2)
        if Mcomp == 'K(M) <= K(f)' or Mcomp == 'K(M) >= K(f)':
            return Mcomp
        elif Mcomp == 'Better lower bound':
            L = M
        elif Mcomp == 'Better upper bound':
            R = M
    
    #Sometimes the MapComparability function continually returns 'Better upper/lower bound' which seems to suggest that the parameter we have is almost exactly at the comparable map,
    #but somehow the program isn't detecting it.  I'm not sure what is going on here, but this is a quick fix.
    if Mcomp == 'Better lower bound':
        return 'K(M) <= K(f)'
    else:
        return 'K(M) >= K(f)'

Now we implement the algorithm for finding the topological entropy of a given map.

In [ ]:
def TopologicalEntropy(q1,q2,N=100,nmax=2500,epsilon=1e-4):
    #This function takes what we have programmed and computes an approximation of the topological entropy of the quartic polynomial f(x) = q2(q1(x)).
    h0 = -0.1
    h1 = np.log(4.1)
    
    for _ in range(nmax):
        #We iterate with a bisection method until the absolute difference is small enough
        h = (h0 + h1)/2
        G = FindComparableMap(q1,q2,h,N)
        
        if G == 'K(M) <= K(f)':
            h0 = h
        elif G == 'K(M) >= K(f)':
            h1 = h
    
        if abs(h1 - h0) < epsilon:
            break

    return h0,h1

Test:

In [97]:
q1 = lambda x : 0.69314718*x*(1-x)
q2 = lambda x : 0.36509305*x*(1-x)

h = TopologicalEntropy(q1,q2)
print(f'The topological entropy of f(x) = q2(q1(x)) is between {h[0]} and {h[1]}')

hdash = TopologicalEntropy(q2,q1)
print(f'The topological entropy of g(x) = q1(q2(x)) is between {hdash[0]} and {hdash[1]}')

The topological entropy of f(x) = q2(q1(x)) is between -0.1 and -0.09990777667396789
The topological entropy of g(x) = q1(q2(x)) is between -0.1 and -0.09990777667396789


In [ ]:
def DrawEntropy(Precision = 100, N=100, nmax=2500,epsilon=1e-4):
    #This function plots the isentropes of of topological entropy within the parameter space of quartic polynomials formed as the composition of two quadratics.

    #construct the parameter space
    X = np.linspace(0,4,Precision+1)
    X1, X2 = np.meshgrid(X,X)
    
    Z = np.zeros((X1.shape[0],X2.shape[0]))

    for ms, m in enumerate(tqdm(X)):
        for ns, n in enumerate(X):
            #compute topological entropy
            q1 = lambda x : m*x*(1-x)
            q2 = lambda x : n*x*(1-x)
            Z[ms,ns] = sum(TopologicalEntropy(q1,q2,N=N,nmax=nmax,epsilon=epsilon))/2
            
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize = (10,10))
    ax.contour(X1,X2,Z,20)

In [100]:
DrawEntropy(Precision = 200, N = 100)

  1%|▏         | 3/201 [00:11<13:01,  3.95s/it]


KeyboardInterrupt: 